In [2]:
from models.model import *
import torch.optim as optim
import data_aggregation.get_features
from models.catboost_model import train_test_split, train_catboost_model, count_metrics
class BugLocalizationModelAPI:
    def __init__(self):
        self.model = None

    def train_grid_search_model(self, embeddings_path='/Users/e.poslovskaya/bug_ml/data/X.npy', labels_path='/Users/e.poslovskaya/bug_ml/data/y.npy', save_path, params=None):
        blm = BugLocalizationModel(embeddings_path, labels_path)
        model = LSTMTagger
        if params is None:
            params = blm.create_list_of_train_hyperparameters()
        blm.train(params, model, top_two=True)
        blm.save_results(name=save_path)

    def fit_model_from_params(self, params=None, use_best_params=False, path_to_results='.', save_path='.'):
        blm = BugLocalizationModel(embeddings_path='./data/X.npy', labels_path='./data/y.npy')
        model = LSTMTagger

        if use_best_params:
            pass # переучиваем модель на лучших параметрах?
        else:
            best_params = params
        blm.train(best_params, model)
        blm.save_model(save_path)
        self.model = blm
        return blm

    def get_code_features(self, methods_data):
        self.feature_extractor = data_aggregation.get_features.FeatureExtractor()
        for method in methods_data:
            self.feature_extractor.get_feature_from_code(method['code'])
            self.feature_extractor.get_feature_from_code(method['meta'])
        return self.feature_extractor.to_pandas()

    def embed_files(self, path_to_files, path_to_methods):
        embeddings = match_embeddings_with_methods.process_data(path_to_files, path_to_methods)
        embeddings = np.array(embeddings).reshape(1, -1, 320)
        return embeddings


    def predict_bug(self, embeddings):
        prediction = self.model.model(FloatTensor(embeddings))[:,:, 1]
        prediction = prediction.flatten()
        return (-prediction).argsort()[:3], prediction


    def fit_catboost(self):
        df_features = self.feature_extractor.to_pandas()
        #df = df.DataFrame({"preds":predict_bug(), "indices":})
        train_pool, test_pool, df_val = train_test_split(df_features)
        model = train_catboost_model(train_pool, test_pool, save=True)

        results = count_metrics(model, df_val)
        self.cb_model = model
        return model
    
api = BugLocalizationModelAPI()
params = Parameters(0.01, 10, optim.Adam, 0.5, 5, 60)
model = api.fit_model_from_params([params], save_path='./data/lstm')



  0%|          | 0/26 [00:00<?, ?it/s]

Parameters(lr=0.01, epoch=10, optim=<class 'torch.optim.adam.Adam'>, anneal_coef=0.5, anneal_epoch=5, dim=60)


[1 / 10] Train: Loss = 0.68975, Accuracy = 39.08%, Previous code mean = 20.00%: 100%|██████████| 26/26 [00:22<00:00,  1.14it/s]
[1 / 10]   Val: Loss = 0.68855, Accuracy = 46.72%, Previous code mean = 48.15%: 100%|██████████| 7/7 [00:02<00:00,  2.85it/s]
[2 / 10] Train: Loss = 0.68808, Accuracy = 52.19%, Previous code mean = 18.75%: 100%|██████████| 26/26 [00:20<00:00,  1.27it/s]
[2 / 10]   Val: Loss = 0.68808, Accuracy = 52.36%, Previous code mean = 41.38%: 100%|██████████| 7/7 [00:02<00:00,  3.06it/s]
[3 / 10] Train: Loss = 0.68765, Accuracy = 55.82%, Previous code mean = 14.00%: 100%|██████████| 26/26 [00:19<00:00,  1.33it/s]
[3 / 10]   Val: Loss = 0.68759, Accuracy = 56.37%, Previous code mean = 45.45%: 100%|██████████| 7/7 [00:02<00:00,  3.22it/s]
[4 / 10] Train: Loss = 0.68735, Accuracy = 59.00%, Previous code mean = 28.00%: 100%|██████████| 26/26 [00:20<00:00,  1.27it/s]
[4 / 10]   Val: Loss = 0.68773, Accuracy = 55.87%, Previous code mean = 45.16%: 100%|██████████| 7/7 [00:02<00

In [1]:
#from code2seq.code2seq import predict
from embeddings import match_embeddings_with_methods
import numpy as np
import sys
import os

PACKAGE_PARENT = '../code2seq'
sys.path.append(os.path.normpath(PACKAGE_PARENT))
PACKAGE_DATA_AGG = '../data_aggregation'
sys.path.append(os.path.normpath(PACKAGE_DATA_AGG))
PACKAGE_DATA_AGG = '../embeddings'
sys.path.append(os.path.normpath(PACKAGE_DATA_AGG))

def embed_files(path_to_files, path_to_methods):
    embeddings = match_embeddings_with_methods.process_data(path_to_files, path_to_methods)
    embeddings = np.array(embeddings).reshape(1, -1, 320)
    return embeddings


def predict_bug(model, embeddings):
    prediction = model.model(FloatTensor(embeddings))[:,:, 1]
    prediction = prediction.flatten()
    return (-prediction).argsort()[:3], prediction


path_to_files = '/Users/e.poslovskaya/bug_ml/ex_reports/4'
path_to_methods = '/Users/e.poslovskaya/bug_ml/ex_reports/stacktrace_ex.json'

embeddings = embed_files(path_to_files, path_to_methods)
predict_bug(model, embeddings)

NameError: name 'model' is not defined